In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.impute import SimpleImputer
lr = 0.3

In [2]:
import pandas as pd
# reading csv files
df =  pd.read_csv('heart.dat', sep=" ", header=None)

In [3]:
num_Interation = 15

In [4]:
column_name = ['age', 'sex', 'chest pain', 'resting blood pressure', 'serum cholestoral', 'fasting blood sugar', 'resting electrocardiographic', 'maximum heart rate achieved',' exercise induced angina', 'oldpeak',' the slope of the peak exercise ST segment', 'number of major vessels', 'tha', 'label']

In [5]:
len(column_name)

14

In [6]:
df.columns = column_name

In [7]:
df

,age,sex,chest pain,resting blood pressure,serum cholestoral,fasting blood sugar,resting electrocardiographic,maximum heart rate achieved,exercise induced angina,oldpeak,the slope of the peak exercise ST segment,number of major vessels,tha,label
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,2
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,1
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,2
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,1
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52.0,1.0,3.0,172.0,199.0,1.0,0.0,162.0,0.0,0.5,1.0,0.0,7.0,1
266,44.0,1.0,2.0,120.0,263.0,0.0,0.0,173.0,0.0,0.0,1.0,0.0,7.0,1
267,56.0,0.0,2.0,140.0,294.0,0.0,2.0,153.0,0.0,1.3,2.0,0.0,3.0,1
268,57.0,1.0,4.0,140.0,192.0,0.0,0.0,148.0,0.0,0.4,2.0,0.0,6.0,1


In [8]:
X = df.iloc[:, :-1]

In [9]:

from sklearn.preprocessing import StandardScaler
trans = StandardScaler()
X = trans.fit_transform(X)

In [10]:
label = df.iloc[:, -1]
#train_date = pd.read_csv('train_date/train_date.csv')

In [11]:
label = label.map({1:0, 2:1})

In [12]:
label = np.array(label)

## Feature important

In [13]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(X)
X_Imputed  = imp_mean.transform(X)

In [14]:
from sklearn.ensemble import RandomForestClassifier

feature_names = [f'feature {i}' for i in range(X_Imputed.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_Imputed, label)
importances = forest.feature_importances_

In [15]:
df_imprtance = pd.DataFrame(importances)

In [16]:
index_top_n_Feature = df_imprtance.sort_values(by=df_imprtance.columns[0],ascending=False).iloc[:12,].index

In [17]:
index_top_n_Feature.values

array([12,  2,  7,  9, 11,  4,  0,  3, 10,  8,  1,  6], dtype=int64)

In [18]:
X_Imputed_df = pd.DataFrame(X_Imputed)

In [19]:
X_Imputed_df = X_Imputed_df.iloc[:,index_top_n_Feature.values]

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow

def Res_model(X, y, index, epoch):
    # define the keras model
    model = Sequential()
    model.add(Dense(248, input_dim=X.shape[-1], activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error'])
    model.fit(X, y, epochs=epoch, batch_size=5000, verbose = 0)
    model.save('DL_model_Heart/model_'+str(index)+'.h5')
    return model.predict(X)

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_Imputed_df, label, test_size=0.3, random_state=0)

In [22]:
from sklearn.metrics import confusion_matrix
# Plot ROC
def roc_plot(y_real, y_predict):    
    from sklearn.metrics import roc_curve, auc    
    fpr, tpr, thresholds = roc_curve(y_real, y_predict)
    auc = auc(fpr, tpr)
    #print('Auc: ',auc)
    #lw = 2
    #plt.figure(1)
    #plt.plot([0, 1], [0, 1], 'k--')
    #plt.plot(fpr, tpr, label='Sleep disorder (area = {:.3f})'.format(auc))
    #plt.plot(fpr, tpr, label='RF (area = {:.3f})'.format(auc))
    #plt.xlabel('False positive rate')
    #plt.ylabel('True positive rate')
    #plt.title('ROC curve')    
    #plt.legend(loc='lower right')
    #plt.show()
    return auc
    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()
    #print('confusion matrix: (tn, fp, fn, tp)',tn, fp, fn, tp)
    #print('acc= ', (tp + tn)/(tp+tn+fp+fn))
    #print('pre= ', tp/(tp+fp))
    #print('sen= Recall= ', (tp)/(tp+fn))
    #print('spec= ', (tn)/(tn+fp))
    #print('F1score= ', 2*(tp)/(2*tp+fp+fn))      
    #print('mcc= ', ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    
    
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    
    
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

In [23]:
# It can be used to reconstruct the model identically.
from tensorflow.keras.models import load_model

def model_Predict(X, num_Interation):
    result = np.array([0.5] * len(X))
    for i in range(0, num_Interation):
        model = load_model('DL_model_Heart/model_'+str(i)+".h5")
        RM_predicted = model.predict(X)
        result = result+lr*RM_predicted.reshape((len(X), ))
    return result

## Running on Multiple parameters

## Running on CLU 33, epoc 200

In [26]:
### from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
# define undersample strategy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import tensorflow
tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

import timeit
import tracemalloc
#from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical



# split 10 fold of undersampling
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve


max_auc = 0.9

for time in range(0, 10):
    rsFile = "Result_OverIteration_Bagging_Numeric/NumberOfIteration"+str(time)+".csv"
    f2=open(rsFile,"w")
    f2.write('num_Interation, epoch, Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    print("working on num_Interation: " + str(time))
    for num_Interation in ([33]): # range(2, 50):
        for epoch in ([190]): # range(10, 500, 10):
            

            Res = []
            predicted_Base =  np.array([0.5] * len(X_train))
            predicted = predicted_Base





            start = timeit.default_timer()
            tracemalloc.start()

            ## Start training
            for i in range(0, num_Interation):
                Res = y_train-predicted
                #print('Res: ',Res)
                Predicted_Res = Res_model(X_train,Res, i, epoch)
                New_predicted = predicted+ lr*Predicted_Res.reshape((Predicted_Res.shape[0],))
                #print('New_predicted: ',New_predicted)
                predicted = New_predicted
                #print('============================')

            current, peak = tracemalloc.get_traced_memory()
            stop = timeit.default_timer()
            Time_train=stop - start
            Train_Peak_RAM = peak / 10**6
            #print('traning time: ', Time_train)
            #print('Peak_Time:', Train_Peak_RAM)





            start = timeit.default_timer()
            tracemalloc.start()

            predicted_test = model_Predict(X_test, num_Interation)

            stop = timeit.default_timer()
            Time_predict=stop - start
            current, peak = tracemalloc.get_traced_memory()
            Peak_Time_prediction = peak / 10**6






            '''result = np.array([[0]*predicted[i].shape[1]]*predicted[i].shape[0])

            for i in range(0, num_Interation):
                #print(result)
                result = result+ predicted[i]
                ##print('*=================')'''




            #print('y_test: ', y_test)
            #print('predicted_test: ', predicted_test)

            auc=roc_auc_score(y_test, predicted_test)
            #print('y_test.shape ' ,y_test.shape)
            #print('==========================================')
            #print(result[:, 1]/num_Interation)
            #print('result.shape ', result.shape)

            if max_auc <= auc:
                #max_auc = auc
                print('===================: ')
                print('======= AUC ==============: ', auc)
                print('=====AUC===AUC=====: ')
                print('===AUC========AUC==: ')
                print('=AUC===========AUC=: ')
                print('=AUC===========AUC=: ')
                print('=AUC===========AUC=: ')
                print('===================: ')
                print('===================: ')

            thres = predicted_test.min()
            while (thres <= predicted_test.max()):
            #for thres in range(1, 1001):
                thres = thres+(predicted_test.max()/500)
                yhat = []
                for i in range(len(predicted_test)):
                    if predicted_test[i]>thres:
                        yhat.append(1)
                    else:
                        yhat.append(0)

                yhat = np.array(yhat)

                tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(y_test, yhat)
                #auc = roc_plot(y_test, yhat)
                f2.write(str(num_Interation)+", "+str(epoch)+", "+str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")


    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       

working on num_Interation: 0
===================: 
======= AUC ==============:  0.9027777777777778
=====AUC===AUC=====: 
===AUC========AUC==: 
=AUC===========AUC=: 
=AUC===========AUC=: 
=AUC===========AUC=: 
===================: 
===================: 
WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!
working on num_Interation: 1
WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!
working on num_Interation: 2
===================: 
======= AUC ==============:  0.9185606060606061
=====AUC===AUC=====: 
===AUC========AUC==: 
=AUC===========AUC=: 
=AUC===========AUC=: 
=AUC===========AUC=: 
===================: 
===================: 
WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!
working on num_Interation: 3
===================: 
======= AUC ==============:  0.9059343434343435
=====AUC===AUC=====: 
===AUC========AUC==: 
=AUC===========AUC=: 
=AUC===========AUC=: 
=AUC===========AUC=: 
===================: 
===================: 
WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
metrics.roc_curve(y_test, clf.predict(X_test), pos_label=2)

In [ ]:
import sklearn.metrics as metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test, clf.predict(X_test), pos_label=2)
metrics.auc(fpr, tpr)

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt


prediction = clf.predict(X_test)

confmat = confusion_matrix(y_true=y_test, y_pred=prediction)
fig, ax = plt.subplots(figsize=(3, 3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i,j], va='center', ha='center')
plt.xlabel('predicted label')        
plt.ylabel('true label')
plt.show()

conf_matrix(y_test, prediction)
roc_plot(y_test, prediction)

In [ ]:
# check xgboost version
import xgboost
print(xgboost.__version__)
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

In [ ]:
# define the model
model = XGBClassifier(scale_pos_weight = len(label[label==0]) / len(label[label==1]), n_estimators=5)
#model = XGBClassifier(scale_pos_weight = 15)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, prediction)

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt


prediction = model.predict(X_test)

confmat = confusion_matrix(y_true=y_test, y_pred=prediction)
fig, ax = plt.subplots(figsize=(3, 3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i,j], va='center', ha='center')
plt.xlabel('predicted label')        
plt.ylabel('true label')
plt.show()

conf_matrix(y_test, prediction)
roc_plot(y_test, prediction)

## Using Deep learning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(32, input_dim=13, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=200, batch_size=512, verbose=1)
# make class predictions with the model
predictions = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt


prediction = predictions.reshape(predictions.shape[0])

confmat = confusion_matrix(y_true=y_test, y_pred=prediction)
fig, ax = plt.subplots(figsize=(3, 3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i,j], va='center', ha='center')
plt.xlabel('predicted label')        
plt.ylabel('true label')
plt.show()

conf_matrix(y_test, prediction)
roc_plot(y_test, prediction)